In [28]:
# 模块导入
import pandas as pd
from multiprocessing import Process, Queue,Lock
from concurrent.futures import ProcessPoolExecutor

from geopy.distance import geodesic
import time

In [29]:
# 读入txt 每一行的数据形如:1,30.624806,104.136604,1,2014/8/3 21:18:46
# 1:出租车ID 2:维度 3:经度 4:载客情况(1为有,0为无) 5:时间
# 读入txt文件，将每个出租车的数据分别存入一个csv文件中
file_path="20140803_train.txt"
column_names=['taxi_id','latitude','longitude','passenger','time']
# 读入txt文件1
df=pd.read_csv(file_path,sep=',',header=None,names=column_names)

# 数据预处理
将GPS轨迹数据转为空驶、载客的一段段轨迹数据，包括起始点、终止点、起止时间、终止时间、载客状态和轨迹长度

存储在`trip_df`中

In [30]:
# 提取出taxi_id为0\1\2\3的数据
df_selected = df[df['taxi_id']<=100].copy() # 测试用
# df_selected = df
# 把time列形如:2014/8/3 06:00:53的数据转换成易于处理的形式
df_selected['time']=pd.to_datetime(df_selected['time'],format='%Y/%m/%d %H:%M:%S')
# 增加一列time_s,表示以秒为单位的时间
df_selected['time_s']=df_selected['time'].dt.hour*3600+df_selected['time'].dt.minute*60+df_selected['time'].dt.second
# 以taxi_id为第一关键字，以time为第二关键字，对数据进行排序
df_selected=df_selected.sort_values(by=['taxi_id','time_s'])
# 删除time列
df_selected=df_selected.drop(['time'],axis=1)
# passenger改为bool型
df_selected['passenger']=df_selected['passenger'].astype('bool')
# 把int64型的数据转换成int32型，减少内存占用
df_selected['taxi_id']=df_selected['taxi_id'].astype('int16') # taxi_id不超过15000，可以用int16
df_selected['time_s']=df_selected['time_s'].astype('int32')
# 把float64型的数据转换成float32型，减少内存占用
df_selected['longitude']=df_selected['longitude'].astype('float32')
df_selected['latitude']=df_selected['latitude'].astype('float32')
# 直接以dataframe的形式保存
df_selected.to_pickle('df_selected.pkl')
# 打印出每一列的数据类型以及各自占用的内存大小
print(df_selected.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 362636 entries, 1261 to 361084
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   taxi_id    362636 non-null  int16  
 1   latitude   362636 non-null  float32
 2   longitude  362636 non-null  float32
 3   passenger  362636 non-null  bool   
 4   time_s     362636 non-null  int32  
dtypes: bool(1), float32(2), int16(1), int32(1)
memory usage: 8.0 MB
None


这里开始保存了df_selected，去concur.py里跑（由于多进程的某些原因，需要在main函数里跑）

下面这个块只是为了分析测试，不用管

In [ ]:
# 每辆车的GPS数据采取的时间间隔最长和最短值
time_interval_min = df_selected.groupby('taxi_id')['time_s'].diff().min()
time_interval_max = df_selected.groupby('taxi_id')['time_s'].diff().max()
print('time_interval_min:',time_interval_min)
print('time_interval_max:',time_interval_max)

# 车辆总数
taxi_num = df_selected['taxi_id'].unique().shape[0]
print('taxi_num:',taxi_num)

# 找出每辆车 时间间隔为time_interval_min的数据
df_test = df_selected.groupby('taxi_id').apply(lambda x:x[x['time_s'].diff()==time_interval_min])
print(df_test)
# 找出每辆车 时间间隔为time_interval_max的数据
df_test = df_selected.groupby('taxi_id').apply(lambda x:x[x['time_s'].diff()==time_interval_max])
print(df_test)

# 数据分析
计算目标:每辆出租车的里程利用率、时间利用率、总空驶时间

存储在`taxi_df`中

计算目标:成都市一天内的打车需求时间分布

可以根据`trip_df`中每单的起始时间，根据24小时进行分布统计，得到一天内的打车需求时间分布

计算目标:成都市一天内的打车需求地区分布